In [1]:
import numpy as np
from ansys.mapdl.core import launch_mapdl

# Launch MAPDL
mapdl = launch_mapdl(loglevel="ERROR")
mapdl.clear()
mapdl.prep7()
mapdl.units("SI")  # Use SI units

# --- Define Materials ---
# Material 1: Concrete (temperature-dependent)
mapdl.mp("DENS", 1, 2400)
mapdl.mptemp(1, 100)
mapdl.mptemp(2, 150)
mapdl.mptemp(3, 200)
mapdl.mptemp(4, 300)
mapdl.mptemp(5, 400)
mapdl.mpdata("C", 1, 1, 900)
mapdl.mpdata("C", 1, 2, 950)
mapdl.mpdata("C", 1, 3, 1000)
mapdl.mpdata("C", 1, 4, 1050)
mapdl.mpdata("C", 1, 5, 1100)
mapdl.mpdata("KXX", 1, 1, 2.5)
mapdl.mpdata("KXX", 1, 2, 2.463)
mapdl.mpdata("KXX", 1, 3, 2.396)
mapdl.mpdata("KXX", 1, 4, 2.259)
mapdl.mpdata("KXX", 1, 5, 2.0)

# Material 2: Structural Steel (assumed constant properties)
mapdl.mp("DENS", 2, 7850)
mapdl.mp("C", 2, 434)
mapdl.mp("KXX", 2, 60.5)

# --- Geometry Parameters ---
od_media = 0.254     # Outer diameter of media (m)
od_hx = 0.0508       # Outer diameter of HX pipe (m)
t_hx = 0.00635       # Wall thickness of pipe (m)
id_hx = od_hx - 2 * t_hx  # Inner diameter of HX pipe (m)
length = 0.508       # Cylinder height
num_radial = 5
num_circ = 20

# --- Create Keypoints ---
k0 = mapdl.k("", 0, 0, 0)
inner_kps = []
outer_kps = []
for i in range(num_circ):
    angle = i * (360 / num_circ)
    x_inner = (id_hx / 2) * np.cos(np.deg2rad(angle))
    y_inner = (id_hx / 2) * np.sin(np.deg2rad(angle))
    x_outer = (od_media / 2) * np.cos(np.deg2rad(angle))
    y_outer = (od_media / 2) * np.sin(np.deg2rad(angle))
    inner_kps.append(mapdl.k("", x_inner, y_inner, 0))
    outer_kps.append(mapdl.k("", x_outer, y_outer, 0))

# --- Create Lines and Areas ---
inner_arcs = [mapdl.l(inner_kps[i], inner_kps[(i+1)%num_circ]) for i in range(num_circ)]
outer_arcs = [mapdl.l(outer_kps[i], outer_kps[(i+1)%num_circ]) for i in range(num_circ)]
radial_lines = [mapdl.l(inner_kps[i], outer_kps[i]) for i in range(num_circ)]
areas = []
for i in range(num_circ):
    next_i = (i+1) % num_circ
    lines = [inner_arcs[i], radial_lines[next_i], outer_arcs[i], radial_lines[i]]
    areas.append(mapdl.al(*lines))

# --- Set element type and mesh controls ---
mapdl.et(1, "SOLID70")
volumes = [mapdl.vext(a, dz=length) for a in areas]

# Set radial mesh size
mapdl.lsel("S", "LINE", "", min(radial_lines), max(radial_lines))
mapdl.lesize("ALL", "", "", num_radial, 1)

# Set axial mesh size
num_z = 10
mapdl.lsel("S", "LOC", "X", 0)
mapdl.lsel("R", "LOC", "Y", 0)
mapdl.lsel("R", "LOC", "Z", 0, length)
mapdl.lesize("ALL", "", "", num_z, 1)
mapdl.allsel()

# --- Assign Material IDs Based on Radius ---
r_inner_pipe = id_hx / 2
r_outer_pipe = od_hx / 2

num_vols = len(areas)  # 9 个区域 → 9 个体积

for i in range(1, num_vols + 1):  # Volume IDs start from 1
    mapdl.vsel("S", "VOLU", "", i)
    mapdl.vsum()

    cx = mapdl.get("CXVAL", "VOLU", 0, "CENT", "X")
    cy = mapdl.get("CYVAL", "VOLU", 0, "CENT", "Y")
    r = np.sqrt(float(cx)**2 + float(cy)**2)

    if r_inner_pipe <= r <= r_outer_pipe:
        mapdl.mat(2)  # Structural steel
        mapdl.vatt(2, 1, 1, 0)
    elif r > r_outer_pipe:
        mapdl.mat(1)  # Concrete
        mapdl.vatt(1, 1, 1, 0)

mapdl.allsel()
mapdl.vmesh("ALL")
mapdl.smrtsize(2)

# --- Plot Mesh ---
mapdl.eplot(vtk=True, show_edges=True, line_width=1)



[82, 87, 110]


Widget(value='<iframe src="http://localhost:52880/index.html?ui=P_0x24c5bcaeab0_0&reconnect=auto" class="pyvis…

In [2]:
mapdl.finish()
mapdl.slashsolu()

# Set Up Transient Analysis
mapdl.antype(4)  # Transient thermal analysis
mapdl.trnopt("FULL")  # Full transient solution
mapdl.kbc(0)  # Ramped loading (smooth boundary conditions)
mapdl.timint("ON")  # Enable time integration

# Define initial condition (uniform temperature)
mapdl.ic("ALL", "TEMP", 100)  # Initial temperature of 100°C

# Define time steps
time_steps = np.linspace(0, 14400, 25)  # 0 to 14400 seconds in 24 steps

temp_load = np.linspace(100, 400, len(time_steps))  # 每一步温度线性上升

for i, t in enumerate(time_steps):
    if i == 0:
        continue  # skip initial step

    mapdl.time(t)


    inner_radius = id_hx / 2
    tol = 0.01 * inner_radius

    mapdl.nsel("S", "EXT", "", inner_radius - tol, inner_radius + tol)
    mapdl.nsel("R", "LOC", "Z", 0, length)
    mapdl.d("ALL", "TEMP", temp_load[i])
    mapdl.nsel("ALL")


    mapdl.autots("ON")
    mapdl.deltim(600)

    mapdl.solve()

mapdl.finish()
mapdl.post1()

*****MAPDL VERIFICATION RUN ONLY*****
     DO NOT USE RESULTS FOR PRODUCTION

          ***** MAPDL RESULTS INTERPRETATION (POST1) *****

 *** NOTE ***                            CP =       0.000   TIME= 00:00:00
 Reading results into the database (SET command) will update the current 
 displacement and force boundary conditions in the database with the     
 values from the results file for that load set.  Note that any          
 subsequent solutions will use these values unless action is taken to    
 either SAVE the current values or not overwrite them (/EXIT,NOSAVE).

In [3]:
import os
import pyvista as pv

output_dir = "temp_frames"
os.makedirs(output_dir, exist_ok=True)

mapdl.post1()
num_steps = len(time_steps)

with open("temperature_output.txt", "w") as f:
    for i in range(1, num_steps):
        mapdl.set(i) 

        mapdl.esel('S', 'MAT', '', 1)  # Select concrete elements (material ID 1)
        mapdl.nsle()  # Select the nodes associated with concrete elements

        temps = mapdl.post_processing.nodal_temperature()  # Get the temperatures for selected nodes

        # If temps are not empty, get min, max, and average
        if temps.any():
            min_temp = np.min(temps)
            max_temp = np.max(temps)
            avg_temp = np.mean(temps)

            # Write min, max, average to a file
            f.writelines(f"Step {i}: min={min_temp}, average={avg_temp}, max={max_temp}\n")
        
        # Create the plot
        mesh = mapdl.mesh.grid
        mesh.point_data["Temperature"] = temps

        plotter = pv.Plotter(off_screen=True, window_size=(800, 600))
        plotter.add_mesh(
            mesh,
            scalars="Temperature",
            cmap="inferno",
            clim=[100, 400],
            show_edges=False
        )

        plotter.add_scalar_bar(title="Temperature (°C)")
        plotter.view_vector((-1, -1, 0.5))
        plotter.screenshot(f"{output_dir}/frame_{i:03d}.png")


In [4]:
import imageio.v2 as imageio

frames = [imageio.imread(f"{output_dir}/frame_{i:03d}.png") for i in range(1, num_steps)]
imageio.mimsave("transient_temperature.gif", frames, duration=0.5)
